In [1]:
import numpy as np # linear algebra
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [2]:
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler

Using TensorFlow backend.


In [3]:
train_file = "dr/train.csv"
test_file = "dr/test.csv"
output_file = "submission_dl.csv"

In [8]:
raw_data = np.loadtxt(train_file, skiprows=1, dtype='int', delimiter=',')

In [9]:
 raw_data[:,1:]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [10]:
raw_data[:,0]

array([1, 0, 1, ..., 7, 6, 9])

In [11]:
x_train, x_val, y_train, y_val = train_test_split(
    raw_data[:,1:], raw_data[:,0], test_size=0.1)

In [12]:
x_train = x_train.reshape(-1, 28, 28, 1)
x_val = x_val.reshape(-1, 28, 28, 1)

In [13]:
x_train = x_train.astype("float32")/255.
x_val = x_val.astype("float32")/255.

In [14]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)

In [15]:
model = Sequential()

model.add(Conv2D(filters = 16, kernel_size = (3, 3), activation='relu',
                 input_shape = (28, 28, 1)))
model.add(BatchNormalization())
model.add(Conv2D(filters = 16, kernel_size = (3, 3), activation='relu'))
model.add(BatchNormalization())
#model.add(Conv2D(filters = 16, kernel_size = (3, 3), activation='relu'))
#model.add(BatchNormalization())
model.add(MaxPool2D(strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation='relu'))
model.add(BatchNormalization())
#model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation='relu'))
#model.add(BatchNormalization())
model.add(MaxPool2D(strides=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [16]:
datagen = ImageDataGenerator(zoom_range = 0.1,
                            height_shift_range = 0.1,
                            width_shift_range = 0.1,
                            rotation_range = 10)

In [17]:
model.compile(loss='categorical_crossentropy', optimizer = Adam(lr=1e-4), metrics=["accuracy"])


In [18]:
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)


In [26]:
hist = model.fit_generator(datagen.flow(x_train, y_train, batch_size=16),
                           steps_per_epoch=700,
                           epochs=30, #Increase this when not on Kaggle kernel
                           verbose=2,  #1 for ETA, 0 for silent
                           validation_data=(x_val[:400,:], y_val[:400,:]), #For speed
                           callbacks=[annealer])

Epoch 1/30
150s - loss: 0.2568 - acc: 0.9429 - val_loss: 0.0601 - val_acc: 0.9800
Epoch 2/30
164s - loss: 0.2168 - acc: 0.9494 - val_loss: 0.0733 - val_acc: 0.9725
Epoch 3/30
150s - loss: 0.1566 - acc: 0.9627 - val_loss: 0.0614 - val_acc: 0.9850
Epoch 4/30
150s - loss: 0.1494 - acc: 0.9624 - val_loss: 0.0304 - val_acc: 0.9875
Epoch 5/30
152s - loss: 0.1281 - acc: 0.9662 - val_loss: 0.0250 - val_acc: 0.9925
Epoch 6/30
148s - loss: 0.1310 - acc: 0.9667 - val_loss: 0.0212 - val_acc: 0.9875
Epoch 7/30
147s - loss: 0.0937 - acc: 0.9756 - val_loss: 0.0208 - val_acc: 0.9950
Epoch 8/30
170s - loss: 0.1100 - acc: 0.9721 - val_loss: 0.0255 - val_acc: 0.9925
Epoch 9/30
221s - loss: 0.0965 - acc: 0.9739 - val_loss: 0.0272 - val_acc: 0.9875
Epoch 10/30
217s - loss: 0.0939 - acc: 0.9752 - val_loss: 0.0264 - val_acc: 0.9900
Epoch 11/30
206s - loss: 0.0759 - acc: 0.9784 - val_loss: 0.0238 - val_acc: 0.9950
Epoch 12/30
180s - loss: 0.0826 - acc: 0.9786 - val_loss: 0.0229 - val_acc: 0.9950
Epoch 13/30
2

In [27]:
final_loss, final_acc = model.evaluate(x_val, y_val, verbose=0)
print("Final loss: {0:.4f}, final accuracy: {1:.4f}".format(final_loss, final_acc))

Final loss: 0.0223, final accuracy: 0.9924


In [28]:
mnist_testset = np.loadtxt(test_file, skiprows=1, dtype='int', delimiter=',')
x_test = mnist_testset.astype("float32")
x_test = x_test.reshape(-1, 28, 28, 1)/255.

In [29]:
y_hat = model.predict(x_test, batch_size=64)


In [30]:
y_pred = np.argmax(y_hat,axis=1)

In [31]:
with open(output_file, 'w') as f :
    f.write('ImageId,Label\n')
    for i in range(len(y_pred)) :
        f.write("".join([str(i+1),',',str(y_pred[i]),'\n']))